In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import torch.nn as nn
import torch
from torch import optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


# ***Competition Overview:***

**Dataset:** The competition dataset comprises approximately 5000 logs of user inputs. These logs are generated while users are composing essays.
**Task:** The goal of this competition is to predict the score that an essay received based on the log of user inputs. The scores are on a scale of 0 to 6, indicating the quality or effectiveness of the essay.
**File and Field Information:**
The competition provides a CSV file called train_logs.csv, which contains the following fields:

* id: This is a unique identifier for each essay.
* event_id: An index indicating the order of events in the log, ordered chronologically.
* down_time: The time when the down event (e.g., keypress or mouse click) occurred, measured in milliseconds.
* up_time: The time when the up event (e.g., key release or mouse release) occurred, measured in milliseconds.
* action_time: The duration of the event, which is the difference between down_time and up_time.
* activity: The category of activity that the event belongs to. It can have values like "Nonproduction," "Input," "Remove/Cut," "Paste," "Replace," or "Move From [x1, y1] To [x2, y2]".
* down_event: The name of the event when the key or mouse is pressed.
* up_event: The name of the event when the key or mouse is released.
* text_change: The text that changed as a result of the event (if any). This field represents the alteration made to the essay text.
* cursor_position: The character index of the text cursor after the event.
* word_count: The word count of the essay after the event.

**Objective:**
Participants in this competition are tasked with using the provided log data to build a predictive model. This model should take the log events as input and predict the essay's score on the 0 to 6 scale.

In [3]:
df = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
df_score = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

In [4]:
test = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')

#### Activity 
* Nonproduction - The event does not alter the text in any way
* Input - The event adds text to the essay
* Remove/Cut - The event removes text from the essay
* Paste - The event changes the text through a paste input
* Replace - The event replaces a section of text with another string
* Move From [x1, y1] To [x2, y2] - The event moves a section of text spanning character index x1, y1 to a new location x2, y2

In [5]:
def move_from(x):
    if 'Move From' in x:
        return 'Move From'
    else:
        return x 

In [6]:
df.activity  = df['activity'].apply(lambda x : move_from(x))

In [7]:
test.activity  = test['activity'].apply(lambda x : move_from(x))

# Creating Essay Dataset

In [8]:
from torch.utils.data import Dataset, DataLoader, random_split

class EssayDataset(Dataset):
    def __init__(self, train_df, score_df):
        self.train_df = train_df
        self.score_df = score_df
        self.unique_ids = train_df['id'].unique()
        self.columns_to_return = ['action_time', 'activity', 'down_event', 'word_count']
        self.down_event_to_index = {event: idx for idx, event in enumerate(unique_down_events)}
        self.unidentified_de_index = self.down_event_to_index.get('Unidentified', len(unique_down_events))
        self.activity_to_index =  {act: idx for idx, act in enumerate(unique_activities)}
        self.unidentified_a_index = self.activity_to_index.get('Unidentified', len(unique_activities))
        self.score_to_index =  {score: idx for idx, score in enumerate(unique_scores)}
        


    def __len__(self):
        return len(self.unique_ids)

    def __getitem__(self, idx):
        essay_id = self.unique_ids[idx]
        essay_data = self.train_df[self.train_df['id'] == essay_id]
        essay_data = essay_data.sort_values(by='event_id')[self.columns_to_return]
        
        # Normalize and replace categories with indices
        essay_data['action_time'] = (essay_data['action_time'] - essay_data['action_time'].min()) / (essay_data['action_time'].max() - essay_data['action_time'].min())
        essay_data['word_count'] = (essay_data['word_count'] - essay_data['word_count'].min()) / (essay_data['word_count'].max() - essay_data['word_count'].min())
        essay_data['down_event'] = essay_data['down_event'].map(self.down_event_to_index).fillna(self.unidentified_de_index).astype(int)
        essay_data['activity'] = essay_data['activity'].map(self.activity_to_index).fillna(self.unidentified_a_index).astype(int)
        
        essay_score = self.score_to_index[self.score_df[self.score_df['id'] == essay_id]['score'].values[0]]
        essay_tensor = torch.Tensor(essay_data.values)
        return essay_tensor, essay_score

In [9]:


unique_down_events = list(df['down_event'].unique())
unique_activities = list(df['activity'].unique())
unique_scores = list(df_score['score'].unique())
print(unique_scores)
unique_activities.append('Unidentified')

dataset = EssayDataset(df, df_score)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

[3.5, 6.0, 2.0, 4.0, 4.5, 2.5, 5.0, 3.0, 1.5, 5.5, 1.0, 0.5]


# Define Input Embeddings


In [10]:
class InputEssayEmbeddings(nn.Module):
    def __init__(self, n_embed):
        super().__init__()

        # Embedding for the categorical columns
        self.embed_activity = nn.Embedding(len(unique_activities), (n_embed-2)//2)
        self.embed_down_event = nn.Embedding(len(unique_down_events), (n_embed-2)//2)
        
        # activities: 6
        # down_event: 131

    def forward(self, input_tensor):
        # Slice columns from the input tensor
        action_time_col = input_tensor[:, :, 0:1]  # [B, num_events, 1]
        activity_col = input_tensor[:, :, 1].long()  # [B, num_events]
        down_event_col = input_tensor[:, :, 2].long()    # [B, num_events]
        word_count_col = input_tensor[:, :, 3:4]  # [B, num_events, 1]

        # Pass through respective embedding layers
        embed_down_event = self.embed_down_event(down_event_col)  # [B, num_events, embed_dim_down_event]
        embed_activity = self.embed_activity(activity_col)        # [B, num_events, embed_dim_activity]
        
        # Concatenate them with the rest of the tensor
        out = torch.cat([action_time_col, embed_activity, embed_down_event, word_count_col], dim=2)
        return out # [B, num_events, n_embed]
    
class PositionalEncoding(nn.Module):
    def __init__(self, n_embed, seq_len):
        super().__init__()
        # Dict size
        self.emb = nn.Embedding(seq_len, n_embed)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)

    def forward(self, x, batched=False):
        """
        :param x: If using batching, should be [batch size, seq len, embedding dim]. Otherwise, [seq len, embedding dim]
        :return: a tensor of the same size with positional embeddings added in
        """
        # Second-to-last dimension will always be sequence length
        input_size = x.shape[-2]
        indices_to_embed = torch.tensor(np.asarray(range(0, input_size))).type(torch.LongTensor).to(self.device)
        if batched:
            # Use unsqueeze to form a [1, seq len, embedding dim] tensor -- broadcasting will ensure that this
            # gets added correctly across the batch
            emb_unsq = self.emb(indices_to_embed).unsqueeze(0)
            return x + emb_unsq
        else:
            return x + self.emb(indices_to_embed)

# Define Architecture Sub Components

In [11]:
class FeedFoward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
        )

    def forward(self, x):
        return self.net(x)
    
class Head(nn.Module):
    def __init__(self, seq_length, n_embed, num_heads, n_internal):
        super().__init__()
        self.K = nn.Linear(n_embed, n_internal)
        self.Q = nn.Linear(n_embed, n_internal)
        self.V = nn.Linear(n_embed, n_internal)
        self.w0 = nn.Linear(n_internal, n_embed // num_heads)
        

    def forward(self, input_vecs):
        keys = self.K(input_vecs) # B, L, d_internal
        d_k = keys.shape[-1]
        queries = self.Q(input_vecs) # B, L, d_internal
        value = self.V(input_vecs) # B, L, d_internal
        
        weights = torch.matmul(queries, keys.transpose(-2, -1)) * d_k**-0.5# L, L
        attention = torch.softmax(weights, dim=-1)

        logit = torch.matmul(attention , value) # B, L, d_internal
        logit = self.w0(logit)
        return logit

class MultiHeadAttention(nn.Module):

    def __init__(self, seq_length, n_embed, num_heads, n_internal):
        super().__init__()
        self.heads = nn.ModuleList([Head(seq_length, n_embed, num_heads, n_internal) for _ in range(num_heads)])
        
        
    def forward(self, input_vecs):
        cls_tokens = []
        for head in self.heads:
            head_out = head(input_vecs)  
            cls_tokens.append(head_out[:, 0])
        cls_tokens_cat = torch.cat(cls_tokens, dim=-1)
        #print(cls_tokens[0].shape)
        return cls_tokens_cat.unsqueeze(1) # B, n_embed

class MHAConvolution(nn.Module):
    def __init__(self, seq_length, n_embed, num_heads, n_internal, stride=1):
        super().__init__()
        self.stride = stride
        self.window_size = seq_length
        self.n_heads = num_heads
        self.cls_token = nn.Parameter(torch.randn(1, 1, n_embed)) 
        self.pos_embedding = PositionalEncoding(n_embed, seq_length)
        self.multi_head_attention = MultiHeadAttention( seq_length+1, n_embed, num_heads, n_internal)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, input_vecs, batched=False): # B, long_seq_len, n_embed
        
        outputs = []
        #print(input_vecs.shape)
        for i in range(0, input_vecs.size(1) - self.window_size + 1, self.stride): # TODO: think about ways to downsample other than stride
            # prepend the cls token to the input
            local_window = input_vecs[:, i:i+self.window_size, :]  # [B, seq_length, embed_size]
            B, L, _ = local_window.size()
            cls_tokens_repeated = self.cls_token.repeat(B, 1, 1) # B, 1, n_embed
            local_window = self.pos_embedding(local_window, batched=batched)
            local_window_cls = torch.cat([cls_tokens_repeated, local_window], dim=1)
            attention_out = self.multi_head_attention(self.ln1(local_window_cls)) # B, 1, n_embed, a single cls vector cated from all heads
            attention_out += cls_tokens_repeated # residual for the cls_tokens
            out = attention_out + self.ffwd(self.ln2(attention_out))
            out = out.view(B, -1) # B, n_heads*n_embed
            outputs.append(out) 
        final = torch.stack(outputs, dim=1)
        #print(final.shape)
        return final # B, (long_seq_len-seq_len)/stride, n_embed
    

# Define the Model

In [12]:
class GlobalAveragingTransformer(nn.Module):
    def __init__(self, seq_len, n_embed, n_internal, num_heads, n_scores):
        super().__init__()
        self.seq_len = seq_len
        self.mha_conv = MHAConvolution(seq_len, n_embed, num_heads, n_internal, stride=seq_len//2)
        self.classifier = nn.Linear(n_embed, n_scores) # consider adding intermediat ffw layers
        self.embedding = InputEssayEmbeddings(n_embed)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, input_tensor, batched=False):
        x = self.embedding(input_tensor)
        x = self.mha_conv(x) # B, down_sampled_len, n_embed
        x_permuted = x.permute(0, 2, 1) # B, n_embed, down_sampled_len
        x = self.adaptive_pool(x_permuted).squeeze(-1) # B, n_embed
        #print(x.shape)
        x = self.classifier(x) # B, n_scores
        if batched:
            return x
        else:
            return x.squeeze(0)

# Create Model

In [13]:
model = GlobalAveragingTransformer(100, 16, 8, 4, len(unique_scores))
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)

model.to(device)


GlobalAveragingTransformer(
  (mha_conv): MHAConvolution(
    (pos_embedding): PositionalEncoding(
      (emb): Embedding(100, 16)
    )
    (multi_head_attention): MultiHeadAttention(
      (heads): ModuleList(
        (0-3): 4 x Head(
          (K): Linear(in_features=16, out_features=8, bias=True)
          (Q): Linear(in_features=16, out_features=8, bias=True)
          (V): Linear(in_features=16, out_features=8, bias=True)
          (w0): Linear(in_features=8, out_features=4, bias=True)
        )
      )
    )
    (ffwd): FeedFoward(
      (net): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=16, bias=True)
      )
    )
    (ln1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (ln2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): Linear(in_features=16, out_features=12, bias=True)
  (embedding): InputEssayEmbeddings(
    (embed_activity): Embedding(

# Run Training Loop

In [17]:
max_epochs = 80
scheduler_enabled = False

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=10)
criterion = torch.nn.CrossEntropyLoss().to(device)
model.train()
dataset_size = len(train_loader)
print("Starting training loop")
print(len(train_loader))
for epoch in range(max_epochs):
    print(f"Training epoch {str(epoch)}")
    num_correct = 0
    num_samples = 0
    i = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        logit = model.forward(batch_x, batched=True)
        #print(logit.shape)
        #print(batch_y.shape)
        loss = criterion(logit, batch_y)
        
        #train_logger.add_scalar('loss', loss, epoch * dataset_size + i)
        _, pred_labels = torch.max(logit, dim=1)
        num_correct += (pred_labels == batch_y).sum().item()
        num_samples += batch_y.size(0)
        
        loss.backward()
        optimizer.step()
        print(f"finished loop {i} out of {dataset_size} with loss {loss}")
        i+=1
        

    acc = 100 * num_correct / num_samples
    print(f"training epoch {epoch} finished with acc {acc}")
    #train_logger.add_scalar('accuracy', acc, epoch * dataset_size + i - 1)
    if scheduler_enabled:
        scheduler.step(acc)
        
    model.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for batch_x, batch_y in validation_data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            logits = model(batch_x)
            _, predicted = torch.max(logits, 1)
            num_samples += batch_y.size(0)
            num_correct += (predicted == batch_y).sum().item()

    acc = 100 * num_correct / num_samples
    print(f"validation epoch {epoch} finished with acc {acc}")
    #valid_logger.add_scalar('accuracy', acc, epoch * dataset_size + i - 1)
    

Starting training loop
1976
Training epoch 0
finished loop 0 out of 1976 with loss 2.876985549926758
finished loop 1 out of 1976 with loss 2.1154582500457764
finished loop 2 out of 1976 with loss 1.8051605224609375
finished loop 3 out of 1976 with loss 3.1828739643096924
finished loop 4 out of 1976 with loss 2.0638580322265625
finished loop 5 out of 1976 with loss 1.8022454977035522
finished loop 6 out of 1976 with loss 2.0413925647735596
finished loop 7 out of 1976 with loss 2.039069652557373
finished loop 8 out of 1976 with loss 1.7704156637191772
finished loop 9 out of 1976 with loss 1.7763490676879883
finished loop 10 out of 1976 with loss 3.6014316082000732
finished loop 11 out of 1976 with loss 2.0008387565612793
finished loop 12 out of 1976 with loss 2.1088063716888428
finished loop 13 out of 1976 with loss 2.0847132205963135
finished loop 14 out of 1976 with loss 3.5835838317871094
finished loop 15 out of 1976 with loss 1.7696325778961182


KeyboardInterrupt: 

In [20]:
df_agg = df.groupby(['id','activity']).agg({
    'event_id': np.max,
    'action_time' : [np.mean, np.sum, np.min, np.max],
    'word_count' : np.max,
    'cursor_position' : np.max
}).reset_index()
df_agg.head(10)
#df_agg.colum1ns = [i+"_"+j for i,j in df_agg.columns]

id       activity event_id action_time                    word_count  \
                               amax        mean     sum amin  amax       amax   
0  001519c8          Input     2556  121.259204  243731   29   248        256   
1  001519c8      Move From     2516    0.000000       0    0     0        256   
2  001519c8  Nonproduction     2557  154.216667   18506    0  2259        256   
3  001519c8     Remove/Cut     2555   81.846523   34130    2   529        256   
4  001519c8        Replace     2482  125.142857     876   98   196        254   
5  0022f953          Input     2359  122.750774  237891   22   250        323   
6  0022f953  Nonproduction     2454   54.255906   13781    0  1758        321   
7  0022f953          Paste     2241   71.000000      71   71    71        316   
8  0022f953     Remove/Cut     2368   90.576923   23550   28   502        322   
9  0022f953        Replace     2325   98.000000      98   98    98        320   

  cursor_position  
             amax  
0            1495  
1             466  
2            1539  
3            1482  
4            1448  
5            1613  
6            1676  
7             696  
8            1597  
9              80

In [ ]:
test = test.groupby(['id','activity']).agg({
    'event_id': np.max,
    'action_time' : [np.mean, np.sum, np.min, np.max],
    'word_count' : np.max,
    'cursor_position' : np.max
}).reset_index()

test.columns = [i+"_"+j for i,j in test.columns]

In [ ]:
feat = ['event_id_amax', 'action_time_mean',
       'action_time_sum', 'action_time_amin', 'action_time_amax',
       'word_count_amax', 'cursor_position_amax']
df_pvt = pd.pivot_table(df_agg, values =feat, index =['id_'],
                         columns =['activity_'], aggfunc = np.max).reset_index()

df_pvt.columns = [i+"_"+j for i,j in df_pvt.columns]

In [ ]:
remaining_cols = list(set(feat) - set(test.columns))
if len(remaining_cols) != 0:
    for i in remaining_cols:
        test[i] = 0

In [ ]:
test = pd.pivot_table(test, values = feat, index =['id_'],
                         columns =['activity_'], aggfunc = np.max).reset_index()

test.columns = [i+"_"+j for i,j in test.columns]

In [ ]:
remaining_cols = list(set(df_pvt.columns) - set(test.columns))
if len(remaining_cols) != 0:
    for i in remaining_cols:
        test[i] = 0

In [ ]:
test = test[df_pvt.columns]

In [ ]:
test = test[df_pvt.columns]
df_pvt = df_pvt.rename(columns={'id__':'id'})
test = test.rename(columns={'id__':'id'})
df_pvt = df_pvt.merge(df_score, on = 'id', how = 'left')

In [ ]:
sns.histplot(df_pvt['score'])
plt.figure(figsize=(15, 7))
sns.heatmap(df_pvt.drop(['id'],axis= 1).corr(), annot=False)

In [ ]:
cols = ['action_time_amax_Input', 'action_time_amax_Move From',
       'action_time_amax_Nonproduction', 'action_time_amax_Paste',
       'action_time_amax_Remove/Cut', 'action_time_amax_Replace',
       'action_time_amin_Input', 'action_time_amin_Move From',
       'action_time_amin_Nonproduction', 'action_time_amin_Paste',
       'action_time_amin_Remove/Cut', 'action_time_amin_Replace',
       'action_time_mean_Input', 'action_time_mean_Move From',
       'action_time_mean_Nonproduction', 'action_time_mean_Paste',
       'action_time_mean_Remove/Cut', 'action_time_mean_Replace',
       'action_time_sum_Input', 'action_time_sum_Move From',
       'action_time_sum_Nonproduction', 'action_time_sum_Paste',
       'action_time_sum_Remove/Cut', 'action_time_sum_Replace',
       'cursor_position_amax_Input', 'cursor_position_amax_Move From',
       'cursor_position_amax_Nonproduction', 'cursor_position_amax_Paste',
       'cursor_position_amax_Remove/Cut', 'cursor_position_amax_Replace',
       'event_id_amax_Input', 'event_id_amax_Move From',
       'event_id_amax_Nonproduction', 'event_id_amax_Paste',
       'event_id_amax_Remove/Cut', 'event_id_amax_Replace',
       'word_count_amax_Input', 'word_count_amax_Move From',
       'word_count_amax_Nonproduction', 'word_count_amax_Paste',
       'word_count_amax_Remove/Cut', 'word_count_amax_Replace',]

In [ ]:
for col in cols:
    print(f'Plots for {col}')
    plt.figure(figsize=(15, 5))

    plt.subplot(1,3, 1)  
    sns.boxplot(y=df_pvt[col], x=df_pvt['score'], color='#4082ed')
    plt.title("Scatterplot with score")

    plt.subplot(1, 3, 2)  
    sns.lineplot(y=df_pvt[col], x=df_pvt['score'], color='#40b9ed')
    plt.title("trend with score")

    plt.subplot(1, 3, 3)  
    sns.histplot(x=df_pvt[col], bins=50, kde=True, color='#40d3ed')
    plt.title(f"Histogram of {col}")

    plt.tight_layout() 
    plt.show()


# XGB model tuned using Optuna

In [ ]:
X = df_pvt.drop(['id','score'], axis = 1)
y = df_pvt['score']

In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'reg:squarederror',  
        'eval_metric': 'rmse',  
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  

best_params = study.best_params
best_rmse = study.best_value

print(f'Best Parameters: {best_params}')
print(f'Best RMSE: {best_rmse}')

final_model = xgb.XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

In [ ]:
print('Train')
print(mean_squared_error(y_train,final_model.predict(X_train)))
print('Test')
print(mean_squared_error(y_test,final_model.predict(X_test)))

In [ ]:
test_pred = final_model.predict(test.drop('id',axis = 1))
test = test[['id']]
test['score'] = test_pred

test.to_csv('submission.csv', index = False)